In [16]:
import os
import sys
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from scipy.special import expit

In [17]:
FIG_FOLDER = 'fig'
SOURCE_FOLDER = os.path.join('data', 'source')
BACKUP_FOLDER = os.path.join('data', 'backup')
print(f"The source folder is: {os.path.abspath(SOURCE_FOLDER)}")
print(f"The figure folder is: {os.path.abspath(FIG_FOLDER)}")
print(f"The backup folder is: {os.path.abspath(BACKUP_FOLDER)}")

# Create folders
for f in SOURCE_FOLDER, FIG_FOLDER, BACKUP_FOLDER:
    os.makedirs(f, exist_ok=True)

The source folder is: /Users/aureliennioche/Documents/PythonProjects/ProspecTonk/data/source
The figure folder is: /Users/aureliennioche/Documents/PythonProjects/ProspecTonk/fig
The backup folder is: /Users/aureliennioche/Documents/PythonProjects/ProspecTonk/data/backup


# Import data

In [18]:
df_bhv = pd.read_csv(os.path.join(BACKUP_FOLDER, "df_bhv.csv"))
df_bhv.date = pd.to_datetime(df_bhv.date)
df_bhv

,monkey,date,c,p0,x0,p1,x1,time_response,left_X,left_Y,...,is_same_x,is_best_left,is_best_right,pair_id,is_control,is_risky,is_neither_risky_nor_control,is_reversed,choose_risky,choose_best
0,Ola,2020-06-25,0,0.75,2,0.75,3,528,1195.0,131.0,...,False,False,True,0,True,False,False,True,False,False
1,Ola,2020-06-25,0,0.50,3,0.25,3,506,1195.0,131.0,...,True,True,False,1,True,False,False,False,False,True
2,Ola,2020-06-25,0,0.75,2,0.75,-2,394,469.0,131.0,...,False,True,False,2,True,False,False,False,False,True
3,Ola,2020-06-25,0,0.25,-2,0.25,-3,396,469.0,131.0,...,False,True,False,3,True,False,False,False,False,True
4,Ola,2020-06-25,0,0.75,-1,0.75,-3,329,469.0,131.0,...,False,True,False,4,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137292,Alv,2020-10-25,0,0.25,-2,1.00,-1,1074,1195.0,131.0,...,False,False,False,83,False,True,False,False,True,False
137293,Alv,2020-10-25,0,1.00,-2,1.00,-3,1995,469.0,131.0,...,False,True,False,30,True,False,False,False,False,True
137294,Alv,2020-10-25,1,0.25,3,0.75,2,843,1195.0,131.0,...,False,False,False,9,False,True,False,False,False,False
137295,Alv,2020-10-25,0,1.00,1,0.25,1,703,469.0,131.0,...,True,True,False,17,True,False,False,False,False,True


## Optimization technique

In [19]:
EPS = np.finfo(float).eps

def objective(param, model, data):
    # Since we will look for the minimum, 
    # let's return -LLS instead of LLS

    inst = model(param=param)

    n = len(data)
    ll = np.zeros(n)

    for i, (_, row) in enumerate(data.iterrows()):
        pi = inst.p_choice(**row)
        ll[i] = np.log(pi + EPS)

    return -ll.sum()

def optimize(model, data):

    # Define an init guess
    init_guess = [(b[1] - b[0])/2 for b in model.fit_bounds]

    # Run the optimizer
    res = scipy.optimize.minimize(
        fun=objective,
        x0=init_guess,
        bounds=model.fit_bounds,
        args=(model, data))

    # Make sure that the optimizer ended up with success
    assert res.success

    # Get the best param and best value from the 
    best_param = res.x
    best_value = res.fun

    return best_param, best_value

## Decision-making model

In [20]:
class Model:
    
    param_labels = ['distortion', 'precision', 'risk_aversion']
    fit_bounds = [(0.2, 1.8), (0.1, 10.0), (-0.99, 0.99)]

    def __init__(self, param):
        self.distortion, self.precision, self.risk_aversion = param
    
    def p_choice(self, p0, x0, p1, x1, c, *args, **kwargs):

        p = self.p(p0=p0, x0=x0, p1=p1, x1=x1)
        return p[int(c)]

    @classmethod
    def softmax(cls, v, precision):
        return expit(v/precision)

    @staticmethod
    def u(x, risk_aversion):
        if isinstance(x, np.ndarray):
            raise Exception
        else:
            if x >= 0:
                return x ** (1-risk_aversion)
            else:
                return - np.abs(x) ** (1 + risk_aversion)
    
    @classmethod
    def pi(cls, p, alpha):
        if isinstance(p, np.ndarray):
            to_return = np.zeros(p.shape)
            unq_zero = p != 0
            to_return[unq_zero] = np.exp(-(-np.log(p)) ** alpha)
            return to_return
        else:
            if p == 0:
                return 0
            else:
                return np.exp(-(-np.log(p)) ** alpha)

    def p(self, p0, x0, p1, x1):

        v0 = self.pi(p0, self.distortion) * self.u(x0, self.risk_aversion)
        v1 = self.pi(p1, self.distortion) * self.u(x1, self.risk_aversion)
        p = np.zeros(2)
        p[0] = self.softmax(v0-v1, self.precision)
        p[1] = 1 - p[0]
        return p

## Trial with Bar

In [21]:
m = "Bar"
cd = "gain" 

# Select the date
df_m = df_bhv[(df_bhv.monkey == m) & (df_bhv.is_risky == True) & (df_bhv[f"is_{cd}"] == True)]

# Drop columns with identical values
nunique = df_m.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 1].index
df_m.drop(cols_to_drop, axis=1)

df_m

,monkey,date,c,p0,x0,p1,x1,time_response,left_X,left_Y,...,is_same_x,is_best_left,is_best_right,pair_id,is_control,is_risky,is_neither_risky_nor_control,is_reversed,choose_risky,choose_best
618,Bar,2020-06-25,1,0.25,3,0.75,1,666,1195.0,131.0,...,False,False,False,88,False,True,False,False,False,False
628,Bar,2020-06-25,1,0.25,3,0.50,2,655,1195.0,131.0,...,False,False,False,43,False,True,False,False,False,False
642,Bar,2020-06-25,1,1.00,1,0.75,3,845,1195.0,131.0,...,False,False,False,15,False,True,False,True,True,False
672,Bar,2020-06-25,1,0.50,3,0.75,2,1374,469.0,131.0,...,False,False,False,81,False,True,False,False,False,False
697,Bar,2020-06-25,1,0.75,1,0.50,2,1094,469.0,131.0,...,False,False,False,12,False,True,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137103,Bar,2020-10-25,1,1.00,1,0.50,2,584,469.0,131.0,...,False,False,False,29,False,True,False,True,True,False
137117,Bar,2020-10-25,1,1.00,1,0.25,2,635,469.0,131.0,...,False,False,False,99,False,True,False,True,True,False
137122,Bar,2020-10-25,0,0.50,3,0.75,1,701,469.0,131.0,...,False,False,False,23,False,True,False,False,True,False
137127,Bar,2020-10-25,1,0.25,3,0.50,1,577,1195.0,131.0,...,False,False,False,95,False,True,False,False,False,False


### Fit all the data at once...

In [22]:
best_param, best_value = optimize(model=Model, data=df_m)
print("best_param", best_param)

best_param [ 0.72158801  0.72756233 -0.20691704]


### Fit the data day by day...

In [23]:
dates = df_m.date.unique()
n_days = len(dates)
n_param = len(Model.param_labels)

r = np.zeros((n_days, n_param))
n_trial = np.zeros(n_days, dtype=int)
for i in tqdm(range(n_days), file=sys.stdout):
    slide = df_m.date == dates[i]
    n_trial[i] = np.sum(slide)
    best_param, best_value = optimize(model=Model, data=df_m[slide])
    r[i] = best_param

df_fit_bar = pd.DataFrame(r, index=dates, columns=Model.param_labels)
df_fit_bar["n"] = n_trial
df_fit_bar

100%|██████████| 113/113 [01:04<00:00,  1.77it/s]


,distortion,precision,risk_aversion,n
2020-06-25,0.831553,1.281043,-0.295716,115
2020-06-26,0.930450,1.079990,-0.067519,138
2020-06-27,0.869438,0.615553,0.111349,116
2020-06-28,0.935447,0.585639,-0.097054,118
2020-06-29,0.744479,0.490433,-0.255992,130
...,...,...,...,...
2020-10-21,0.618826,0.223102,0.466522,14
2020-10-22,0.606719,0.100000,-0.135914,12
2020-10-23,1.261552,0.100000,-0.289819,12
2020-10-24,0.309839,0.251927,0.398676,6


### Fit the data using a window

In [24]:
w = 10

dates = df_m.date.unique()
n_days = len(dates)
n_param = len(Model.param_labels)

r = np.zeros((n_days, n_param))
n_trial = np.zeros(n_days, dtype=int)
for i in tqdm(range(n_days), file=sys.stdout):
    
    # Apply mask for dates
    b0 = dates[max(i-(w-1), 0)]
    b1 = dates[min(i+w, n_days-1)]
    mask = (df_m.date >= b0) & (df_m.date <= b1)
    
    # Optimize
    best_param, best_value = optimize(model=Model, data=df_m[mask])
    r[i] = best_param
    
    # Count thre trials
    n_trial[i] = np.sum(mask)

df_fit_bar_window = pd.DataFrame(r, index=dates, columns=Model.param_labels)
df_fit_bar_window["n"] = n_trial
df_fit_bar_window

100%|██████████| 113/113 [16:45<00:00,  8.90s/it]


,distortion,precision,risk_aversion,n
2020-06-25,0.861389,0.887937,-0.260099,855
2020-06-26,0.834342,0.865360,-0.257379,873
2020-06-27,0.802772,0.872041,-0.281786,902
2020-06-28,0.795199,0.874207,-0.281777,914
2020-06-29,0.801317,0.876549,-0.282040,926
...,...,...,...,...
2020-10-21,0.559043,0.270883,0.069137,160
2020-10-22,0.568058,0.273317,0.092846,149
2020-10-23,0.586313,0.281441,0.111589,137
2020-10-24,1.530327,0.615438,0.144181,125


## Day by day (with all the monkeys)

### Do the fit

In [28]:
df_fit_day = pd.DataFrame()

cond = "gain", "loss"
monkeys = df_bhv.monkey.unique()

for i_m, m in enumerate(monkeys):

    print(f"monkey {i_m+1}/{len(monkeys)}")

    for cd in cond:
        
        # Select the data
        df_m = df_bhv[(df_bhv.monkey == m) & (df_bhv.is_risky == True) & (df_bhv[f"is_{cd}"] == True)]
        
        # Get the dates
        dates = df_m.date.unique()
        n_days = len(dates)
        
        # Get the number of parameters
        n_param = len(Model.param_labels)
        
        # Optimize for each day separately
        r = np.zeros((n_days, n_param))
        n_trial = np.zeros(n_days, dtype=int)
        for i in tqdm(range(n_days), file=sys.stdout):
            slide = df_m.date == dates[i]
            n_trial[i] = np.sum(slide)
            best_param, best_value = optimize(model=Model, data=df_m[slide])
            r[i] = best_param
        
        # Backup
        df_m_cd = pd.DataFrame(r, columns=Model.param_labels)
        df_m_cd["date"] = dates
        df_m_cd["monkey"] = m
        df_m_cd["condition"] = cd.replace("is_", "")
        df_m_cd["n"] = n_trial
        df_fit_day = pd.concat((df_fit_day, df_m_cd))

df_fit_day.to_csv(os.path.join(BACKUP_FOLDER, "df_fit_day.csv"), index=False)

monkey 1/15
100%|██████████| 86/86 [00:17<00:00,  5.01it/s]
monkey 2/15
100%|██████████| 103/103 [00:52<00:00,  1.98it/s]
monkey 3/15
100%|██████████| 60/60 [00:25<00:00,  2.39it/s]
monkey 4/15
100%|██████████| 113/113 [00:54<00:00,  2.06it/s]
monkey 5/15
100%|██████████| 66/66 [00:09<00:00,  6.61it/s]
monkey 6/15
100%|██████████| 111/111 [00:46<00:00,  2.37it/s]
monkey 7/15
100%|██████████| 113/113 [01:08<00:00,  1.65it/s]
monkey 8/15
100%|██████████| 106/106 [00:37<00:00,  2.80it/s]
monkey 9/15
100%|██████████| 98/98 [00:27<00:00,  3.50it/s]
monkey 10/15
100%|██████████| 73/73 [00:13<00:00,  5.46it/s]
monkey 11/15
100%|██████████| 89/89 [00:15<00:00,  5.60it/s]
monkey 12/15
100%|██████████| 80/80 [00:20<00:00,  3.82it/s]
monkey 13/15
100%|██████████| 70/70 [00:23<00:00,  2.94it/s]
monkey 14/15
100%|██████████| 78/78 [00:44<00:00,  1.76it/s]
monkey 15/15
100%|██████████| 62/62 [00:07<00:00,  7.94it/s]


### Load the results

In [30]:
df_fit_day = pd.read_csv(os.path.join(BACKUP_FOLDER, "df_fit_day.csv"))
df_fit_day.date = pd.to_datetime(df_fit_day.date)
df_fit_day

,distortion,precision,risk_aversion,date,monkey,condition,n
0,0.200000,10.000000,0.990000,2020-06-25,Ola,gain,46
1,1.800000,2.298505,0.095405,2020-06-26,Ola,gain,13
2,1.800000,0.641667,0.929759,2020-06-27,Ola,gain,18
3,0.200000,0.487869,0.004734,2020-06-28,Ola,gain,12
4,1.192238,0.917417,-0.192953,2020-06-29,Ola,gain,23
...,...,...,...,...,...,...,...
2607,0.200000,10.000000,0.990000,2020-10-20,Yin,loss,3
2608,0.200000,1.038210,-0.879448,2020-10-21,Yin,loss,6
2609,0.784174,0.100000,-0.990000,2020-10-22,Yin,loss,6
2610,1.800000,1.078414,0.873188,2020-10-24,Yin,loss,6


## Using a slide window (with all the monkeys)

### Dot the fit

In [32]:
# How many days before and after?
w = 10

df_fit = pd.DataFrame()

cond = "gain", "loss"
monkeys = df_bhv.monkey.unique()

for i_m, m in enumerate(monkeys):

    print(f"monkey {i_m+1}/{len(monkeys)}")

    for cd in cond:
        
        # Get data
        df_m = df_bhv[(df_bhv.monkey == m) & (df_bhv.is_risky == True) & (df_bhv[f"is_{cd}"] == True)]
        
        # Get dates
        dates = df_m.date.unique()
        n_days = len(dates)
        n_param = len(Model.param_labels)
        
        # Optimize for each 'windowed' day
        r = np.zeros((n_days, n_param))
        n_trial = np.zeros(n_days, dtype=int)
        for i in tqdm(range(n_days), file=sys.stdout):
            
            # Get the mask
            b0 = dates[max(i-w, 0)]
            b1 = dates[min(i+w, n_days-1)]
            mask = (df_m.date >= b0) & (df_m.date <= b1)
            
            # Optimize
            best_param, best_value = optimize(model=Model, data=df_m[mask])
            r[i] = best_param
            
            # Count thre trials
            n_trial[i] = np.sum(mask)
        
        # Backup
        df_m_cd = pd.DataFrame(r, columns=Model.param_labels)
        df_m_cd["date"] = dates
        df_m_cd["monkey"] = m
        df_m_cd["condition"] = cd
        df_m_cd["n"] = n_trial
        df_fit = pd.concat((df_fit, df_m_cd))

df_fit.to_csv(os.path.join(BACKUP_FOLDER, f"df_fit_w{w}.csv"))

monkey 1/15
100%|██████████| 86/86 [04:04<00:00,  2.85s/it]
monkey 2/15
100%|██████████| 103/103 [14:42<00:00,  8.56s/it]
monkey 3/15
100%|██████████| 60/60 [06:43<00:00,  6.72s/it]
monkey 4/15
100%|██████████| 113/113 [17:07<00:00,  9.09s/it]
monkey 5/15
100%|██████████| 66/66 [02:31<00:00,  2.29s/it]
monkey 6/15
100%|██████████| 111/111 [13:42<00:00,  7.41s/it]
monkey 7/15
100%|██████████| 113/113 [20:55<00:00, 11.11s/it]
monkey 8/15
100%|██████████| 106/106 [10:55<00:00,  6.18s/it]
monkey 9/15
100%|██████████| 98/98 [09:09<00:00,  5.61s/it]
monkey 10/15
100%|██████████| 73/73 [04:26<00:00,  3.66s/it]
monkey 11/15
100%|██████████| 89/89 [05:09<00:00,  3.48s/it]
monkey 12/15
100%|██████████| 80/80 [06:35<00:00,  4.95s/it]
monkey 13/15
100%|██████████| 70/70 [06:48<00:00,  5.83s/it]
monkey 14/15
100%|██████████| 78/78 [15:31<00:00, 11.95s/it]
monkey 15/15
100%|██████████| 62/62 [01:10<00:00,  1.14s/it]


### Load the results

In [35]:
df_fit = pd.read_csv(os.path.join(BACKUP_FOLDER, f"df_fit_w{w}.csv"), index_col=0)
df_fit.date = pd.to_datetime(df_fit.date)
df_fit

,distortion,precision,risk_aversion,date,monkey,condition,n
0,0.200000,4.270073,0.990000,2020-06-25,Ola,gain,184
1,0.200000,2.770975,0.371047,2020-06-26,Ola,gain,190
2,0.200000,3.808986,0.430657,2020-06-27,Ola,gain,208
3,0.200000,3.722515,0.085129,2020-06-28,Ola,gain,220
4,0.200000,4.670508,0.132597,2020-06-29,Ola,gain,244
...,...,...,...,...,...,...,...
57,0.200000,10.000000,-0.990000,2020-10-20,Yin,loss,77
58,0.313026,10.000000,-0.990000,2020-10-21,Yin,loss,71
59,0.200000,10.000000,-0.990000,2020-10-22,Yin,loss,70
60,1.159000,10.000000,0.790311,2020-10-24,Yin,loss,66


# At once

In [ ]:
cond = "gain", "loss"
monkeys = df_bhv.monkey.unique()

df_fit_overall = pd.DataFrame()

for i_m, m in tqdm(enumerate(monkeys), file=sys.stdout, total=len(monkeys)):

    for cd in cond:
        
        # Select the data
        df_m = df_bhv[(df_bhv.monkey == m) & (df_bhv.is_risky == True) & (df_bhv[f"is_{cd}"] == True)]
        
        # Get the dates
        dates = df_m.date.unique()
        
        # Get the number of parameters
        n_param = len(Model.param_labels)
        
        # Optimize
        best_param, best_value = optimize(model=Model, data=df_m)
        
        # Backup
        df_fit_m = pd.DataFrame(best_param.reshape((1, len(best_param))), columns=Model.param_labels)
        df_fit_m["date_begin"] = dates[0]
        df_fit_m["date_end"] = dates[-1]
        df_fit_m["monkey"] = m
        df_fit_m["condition"] = cd
        df_fit_m["n"] = len(df_m)
        df_fit_overall = pd.concat((df_fit_overall, df_fit_m))
        
df_fit_overall.to_csv(os.path.join(BACKUP_FOLDER, f"df_fit_overall.csv"))

 33%|███▎      | 5/15 [04:38<07:36, 45.66s/it]

### Load the results

In [ ]:
df_fit_overall = pd.read_csv(os.path.join(BACKUP_FOLDER, f"df_fit_overall.csv"), index_col=0)
df_fit_overall.date = pd.to_datetime(df_fit.date)
df_fit_overall